# 🚦 BRSSD Traffic Sign Detection with YOLOv10
## Bangladeshi Road Sign Symbol Dataset Training

This notebook trains YOLOv10 on the BRSSD dataset using Google Colab's free GPU.

### Setup Instructions:
1. **Enable GPU**: Go to `Runtime` → `Change runtime type` → Select `GPU` (T4 recommended)
2. **Run all cells** in order
3. **Download trained model** at the end

---

## 📦 Step 1: Install Dependencies

In [ ]:
# Install required packages
!pip install ultralytics roboflow -q

# Verify GPU availability
import torch
print(f"\n{'='*60}")
print("GPU Setup Verification")
print(f"{'='*60}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("⚠️ WARNING: GPU not available! Please enable GPU in Runtime settings.")
print(f"{'='*60}\n")

## 📥 Step 2: Download BRSSD Dataset from Roboflow

In [ ]:
from roboflow import Roboflow
import os

# Your Roboflow API key
API_KEY = "8GeCUXQU6JPzxMaYOe4m"  # Replace with your API key if different

# Initialize Roboflow
rf = Roboflow(api_key=API_KEY)

# Download the BRSSD dataset
print("Downloading BRSSD dataset from Roboflow...")
project = rf.workspace("mostafinafis").project("road-sign-detection-in-bd")
dataset = project.version(1).download("yolov8")

print(f"\n✓ Dataset downloaded to: {dataset.location}")
print(f"\nDataset structure:")
!ls -lh {dataset.location}

## 🔍 Step 3: Explore Dataset

In [ ]:
import yaml
from pathlib import Path

# Load and display dataset configuration
data_yaml_path = f"{dataset.location}/data.yaml"
with open(data_yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)

print("Dataset Configuration:")
print(f"{'='*60}")
print(f"Number of classes: {data_config['nc']}")
print(f"\nClass names:")
for i, name in enumerate(data_config['names']):
    print(f"  {i}: {name}")

# Count images
dataset_path = Path(dataset.location)
train_imgs = len(list((dataset_path / 'train/images').glob('*.*')))
val_imgs = len(list((dataset_path / 'valid/images').glob('*.*')))
test_imgs = len(list((dataset_path / 'test/images').glob('*.*')))

print(f"\nDataset Statistics:")
print(f"{'='*60}")
print(f"Training images: {train_imgs}")
print(f"Validation images: {val_imgs}")
print(f"Test images: {test_imgs}")
print(f"Total images: {train_imgs + val_imgs + test_imgs}")
print(f"{'='*60}")

## 🖼️ Step 4: Visualize Sample Images

In [ ]:
import cv2
import matplotlib.pyplot as plt
import random
from pathlib import Path

def visualize_samples(dataset_path, split='train', num_samples=6):
    """Visualize random samples from the dataset with labels"""
    img_dir = Path(dataset_path) / split / 'images'
    label_dir = Path(dataset_path) / split / 'labels'
    
    img_files = list(img_dir.glob('*.jpg')) + list(img_dir.glob('*.png'))
    samples = random.sample(img_files, min(num_samples, len(img_files)))
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()
    
    for idx, img_path in enumerate(samples):
        # Load image
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]
        
        # Load corresponding label
        label_path = label_dir / (img_path.stem + '.txt')
        if label_path.exists():
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        cls_id, x_center, y_center, width, height = map(float, parts[:5])
                        
                        # Convert YOLO format to pixel coordinates
                        x1 = int((x_center - width/2) * w)
                        y1 = int((y_center - height/2) * h)
                        x2 = int((x_center + width/2) * w)
                        y2 = int((y_center + height/2) * h)
                        
                        # Draw bounding box
                        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(img, str(int(cls_id)), (x1, y1-5),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        axes[idx].imshow(img)
        axes[idx].set_title(f"{img_path.name}")
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualize training samples
print("Sample Training Images with Annotations:")
visualize_samples(dataset.location, 'train', 6)

## 🚀 Step 5: Train YOLOv10 Model

You can choose different model sizes:
- **YOLOv10n** (nano): Fastest, smallest
- **YOLOv10s** (small): Good balance
- **YOLOv10m** (medium): Better accuracy
- **YOLOv10l** (large): High accuracy
- **YOLOv10x** (xlarge): Best accuracy, slower

In [ ]:
from ultralytics import YOLO

# Training configuration
MODEL_SIZE = 'n'  # Change to 's', 'm', 'l', or 'x' for different sizes
EPOCHS = 100       # Number of epochs
BATCH_SIZE = 16    # Batch size (adjust based on GPU memory)
IMG_SIZE = 640     # Image size

# Initialize model
model = YOLO(f'yolov10{MODEL_SIZE}.pt')

print(f"\n{'='*60}")
print(f"Training YOLOv10-{MODEL_SIZE.upper()} on BRSSD Dataset")
print(f"{'='*60}")
print(f"Model: yolov10{MODEL_SIZE}.pt")
print(f"Epochs: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Image size: {IMG_SIZE}")
print(f"Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")
print(f"{'='*60}\n")

# Train the model
results = model.train(
    data=data_yaml_path,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    name=f'YOLOv10{MODEL_SIZE}_BRSSD_GPU',
    patience=50,
    save=True,
    device=0,  # Use GPU
    workers=8,
    project='runs/detect',
    exist_ok=True,
    pretrained=True,
    optimizer='auto',
    verbose=True,
    seed=42,
    deterministic=True,
    val=True,
    plots=True,
    
    # Data augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    shear=0.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.0,
)

print("\n✓ Training completed!")

## 📊 Step 6: Evaluate Model Performance

In [ ]:
# Validate the model
print("Running validation on test set...")
metrics = model.val()

print(f"\n{'='*60}")
print("Validation Metrics")
print(f"{'='*60}")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")
print(f"{'='*60}\n")

## 📈 Step 7: Visualize Training Results

In [ ]:
from IPython.display import Image, display

# Display training results
results_dir = f'runs/detect/YOLOv10{MODEL_SIZE}_BRSSD_GPU'

print("Training Curves:")
display(Image(filename=f'{results_dir}/results.png', width=800))

print("\nConfusion Matrix:")
display(Image(filename=f'{results_dir}/confusion_matrix.png', width=800))

print("\nPR Curve:")
display(Image(filename=f'{results_dir}/PR_curve.png', width=800))

print("\nF1 Curve:")
display(Image(filename=f'{results_dir}/F1_curve.png', width=800))

## 🎯 Step 8: Test Model on Sample Images

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

# Load best model
best_model = YOLO(f'{results_dir}/weights/best.pt')

# Get test images
test_img_dir = Path(dataset.location) / 'test/images'
test_images = list(test_img_dir.glob('*.jpg'))[:6]  # First 6 test images

# Run predictions
print("Running predictions on test images...\n")
results_list = best_model.predict(test_images, conf=0.25, save=True)

# Display predictions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, (img_path, result) in enumerate(zip(test_images, results_list)):
    img = result.plot()  # Plot predictions on image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    axes[idx].imshow(img)
    axes[idx].set_title(f"{img_path.name}")
    axes[idx].axis('off')

plt.tight_layout()
plt.suptitle('Model Predictions on Test Images', fontsize=16, y=1.02)
plt.show()

print("\n✓ Predictions completed!")

## 💾 Step 9: Download Trained Model

In [ ]:
from google.colab import files
import shutil

# Create a zip file with the trained model and results
output_zip = f'YOLOv10{MODEL_SIZE}_BRSSD_trained'
shutil.make_archive(output_zip, 'zip', results_dir)

print(f"Model and results archived to: {output_zip}.zip")
print(f"\nBest model weights: {results_dir}/weights/best.pt")
print(f"Last model weights: {results_dir}/weights/last.pt")

# Download the best weights
print("\nDownloading best model weights...")
files.download(f'{results_dir}/weights/best.pt')

# Optionally download the entire results folder
print("\nTo download all results (training curves, confusion matrix, etc.):")
print(f"Uncomment the line below to download {output_zip}.zip")
# files.download(f'{output_zip}.zip')

print("\n✓ Model download complete!")

## 🔄 Step 10: Export Model (Optional)

Export the model to different formats for deployment.

In [ ]:
# Load best model
best_model = YOLO(f'{results_dir}/weights/best.pt')

# Export to ONNX format (for wider compatibility)
print("Exporting to ONNX format...")
best_model.export(format='onnx')

# Export to TensorFlow Lite (for mobile deployment)
# print("Exporting to TensorFlow Lite...")
# best_model.export(format='tflite')

# Export to TensorRT (for NVIDIA GPUs)
# print("Exporting to TensorRT...")
# best_model.export(format='engine')

print("\n✓ Model export complete!")

## 📝 Step 11: Save Training Summary

In [ ]:
import json
from datetime import datetime

# Create training summary
summary = {
    'model': f'YOLOv10{MODEL_SIZE}',
    'dataset': 'BRSSD (Bangladeshi Road Sign Symbol Dataset)',
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'epochs': EPOCHS,
    'batch_size': BATCH_SIZE,
    'image_size': IMG_SIZE,
    'num_classes': data_config['nc'],
    'class_names': data_config['names'],
    'training_images': train_imgs,
    'validation_images': val_imgs,
    'test_images': test_imgs,
    'metrics': {
        'mAP50': float(metrics.box.map50),
        'mAP50-95': float(metrics.box.map),
        'precision': float(metrics.box.mp),
        'recall': float(metrics.box.mr)
    },
    'device': 'GPU' if torch.cuda.is_available() else 'CPU',
    'weights_path': f'{results_dir}/weights/best.pt'
}

# Save summary to JSON
summary_path = f'{results_dir}/training_summary.json'
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)

# Display summary
print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)
print(json.dumps(summary, indent=2))
print("="*60)

print(f"\n✓ Training summary saved to: {summary_path}")

## 🎉 Training Complete!

Your YOLOv10 model has been successfully trained on the BRSSD dataset!

### Next Steps:
1. ✅ Download the trained model weights (`best.pt`)
2. ✅ Review the training metrics and visualizations
3. ✅ Test the model on your own images
4. ✅ Deploy the model in your application

### Model Files:
- **Best weights**: `runs/detect/YOLOv10{MODEL_SIZE}_BRSSD_GPU/weights/best.pt`
- **Last weights**: `runs/detect/YOLOv10{MODEL_SIZE}_BRSSD_GPU/weights/last.pt`
- **Training results**: `runs/detect/YOLOv10{MODEL_SIZE}_BRSSD_GPU/`

### Using the Trained Model:
```python
from ultralytics import YOLO

# Load your trained model
model = YOLO('path/to/best.pt')

# Run inference
results = model.predict('path/to/image.jpg', conf=0.25)

# Display results
results[0].show()
```

---
**Happy detecting! 🚦**